In [91]:
#dependenices
import pandas as pd

from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup

import requests
import pymongo
import time
import json

In [48]:
def init_browser():
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [75]:
# open browser window using splinter 
def init_visit_web(url):
    executable_path = {"executable_path":"chromedriver.exe"}
    browser=Browser("chrome", **executable_path, headless=False)
    browser.visit(url)
    # wait for page to be load
    time.sleep(1)
    soup = BeautifulSoup(browser.html, 'lxml')
    return soup

# NASA MARS NEWS

In [132]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
soup=init_visit_web(url)

In [105]:
# get list of elements to extract the data from
result=soup.find('div', class_='list_text')
news_title=result.find('div', class_='content_title',recursive=True).text
news_p=result.find('div', class_='article_teaser_body',recursive=True).text

In [106]:
print(news_title)
print(news_p)

Space History Is Made in This NASA Robot Factory
From rockets to rovers, JPL's Spacecraft Assembly Facility has been at the center of robotic spaceflight. Here's a closer look at what makes it so special.


# JPL Mars Space Images - Featured Image

In [61]:
browser=init_browser()
url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [62]:
browser.click_link_by_id('full_image')
time.sleep(3)

In [63]:
# click the button opening details page for the featured image
browser.click_link_by_partial_text('more info')

C:\Users\16478\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:504: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [67]:
soup = BeautifulSoup(browser.html, 'lxml')
# Parse HTML with Beautiful Soup
img_url = soup.find('figure', class_='lede').a['href']
print(img_url)
featured_image_url='https://www.jpl.nasa.gov'+ img_url
print(featured_image_url)

/spaceimages/images/largesize/PIA18291_hires.jpg
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18291_hires.jpg


# Mars Weather

In [77]:
url = 'https://twitter.com/marswxreport?lang=en'
soup = init_visit_web(url)

In [78]:
mars_weather = soup.find('p', class_='TweetTextSize').text
print(mars_weather)

InSight sol 380 (2019-12-21) low -97.9ºC (-144.3ºF) high -20.6ºC (-5.0ºF)
winds from the SSE at 6.3 m/s (14.1 mph) gusting to 21.5 m/s (48.2 mph)
pressure at 6.50 hPa


# Mars Facts

In [79]:
url = 'https://space-facts.com/mars/s'
tables = pd.read_html(url)
tables[0]

,0,1
0,Equatorial Diameter:,"120,536 km"
1,Polar Diameter:,"108,728 km"
2,Mass:,5.68 × 10^26 kg (95 Earths)
3,Moons:,"82 (Titan, Enceladus, Iapetus & Rhea)"
4,Rings:,30+ (7 Groups)
5,Orbit Distance:,"1,426,666,422 km (9.58 AU)"
6,Orbit Period:,"10,756 days (29.5 years)"
7,Surface Temperature:,-139 °C
8,First Record:,8th century BC
9,Recorded By:,Assyrians


In [80]:
df = tables[0]
df.columns = ['Parameter', 'Value']
df.set_index('Parameter',inplace=True)
mars_facts = df.to_html().replace('\n', '')
mars_facts

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>120,536 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>108,728 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>5.68 × 10^26 kg (95 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>82 (Titan, Enceladus, Iapetus &amp; Rhea)</td>    </tr>    <tr>      <th>Rings:</th>      <td>30+ (7 Groups)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>1,426,666,422 km (9.58 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>10,756 days (29.5 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-139 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>8th\xa0century BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Assyrians</td>    </tr>  </tbody></table>'

# Mars Hemispheres

In [151]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
soup = init_visit_web(url)

In [152]:
#extract links to Mars Hemispehere images
results=soup.find_all("div", class_="item")

In [153]:
hemisphere_image_urls = []
for result in results:
    title = result.find("h3").text
    title = title.replace("Enhanced", "")
    end_url = result.find("a")["href"]
    img_url = "https://astrogeology.usgs.gov" + end_url  
    print(img_url)
    soup = init_visit_web(img_url)
    image_div = soup.find("div", class_="downloads")
    image_url = image_div.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})
print(hemisphere_image_urls)

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
[{'title': 'Cerberus Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [146]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [147]:
db = client.mission_to_mars_db
collection = db.items
db.items.delete_many({})

In [115]:
hemisphere_image_url=json.dumps(hemisphere_image_urls)

In [116]:
print(hemisphere_image_url)

[{"title": "Cerberus Hemisphere ", "img_url": "http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"}, {"title": "Schiaparelli Hemisphere ", "img_url": "http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg"}, {"title": "Syrtis Major Hemisphere ", "img_url": "http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg"}, {"title": "Valles Marineris Hemisphere ", "img_url": "http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg"}]


In [118]:
 mars_record= {
            'news_title': news_title,
            'news_p': news_p,
            'featured_image_url': featured_image_url,
            'mars_weather': mars_weather,
            'mars_facts': mars_facts,
            'hemisphere_image_urls': hemisphere_image_url
        }

In [144]:
print(mars_record)

{'news_title': 'Space History Is Made in This NASA Robot Factory', 'news_p': "From rockets to rovers, JPL's Spacecraft Assembly Facility has been at the center of robotic spaceflight. Here's a closer look at what makes it so special.", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18291_hires.jpg', 'mars_weather': 'InSight sol 380 (2019-12-21) low -97.9ºC (-144.3ºF) high -20.6ºC (-5.0ºF)\nwinds from the SSE at 6.3 m/s (14.1 mph) gusting to 21.5 m/s (48.2 mph)\npressure at 6.50 hPa', 'mars_facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>120,536 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>108,728 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>5.68 × 10^26 kg (95 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <

In [148]:
# Insert data into Mongo DB
db.items.insert_one(mars_record)

In [150]:
cursor = db.mission_to_mars_db # choosing the collection you need
for document in cursor.find():
    print (document)